In [1]:
import sys
import os

# Add src folder to path (we're in notebooks/, src is one level up)
sys.path.append('src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import our modules
from data_loader import TennisDataLoader, load_and_prepare_data
from elo_calculator import calculate_elo_for_dataframe
from feature_engineering import engineer_all_features
from model import TennisPredictionModel
from visualizations import *

print("✅ All modules imported!")
print("\n🎾 STARTING COMPLETE TENNIS PREDICTION PIPELINE")
print("="*60)

✅ All modules imported!

🎾 STARTING COMPLETE TENNIS PREDICTION PIPELINE


In [2]:
print("\n📂 STEP 1: LOADING & CLEANING DATA")
print("="*60)

# Load data
loader = TennisDataLoader('data/raw/atp_tennis.csv')
df = loader.load_data()
df = loader.clean_data()
loader.get_data_summary()

# Split into train/test
train_df, test_df = loader.split_train_test(test_year=2025)

print(f"\n📊 Training: {len(train_df):,} matches (2000-2024)")
print(f"📊 Test: {len(test_df):,} matches (2025)")

# Save cleaned data
import os
os.makedirs('../data/processed', exist_ok=True)
train_df.to_csv('../data/processed/train_2000_2024.csv', index=False)
test_df.to_csv('../data/processed/test_2025.csv', index=False)
print("\n✅ Cleaned data saved!")


📂 STEP 1: LOADING & CLEANING DATA
Loading data from data/raw/atp_tennis.csv...
✅ Loaded 66,681 matches

🧹 Cleaning data...
✅ Data cleaned. 66,681 matches remaining

📊 DATA SUMMARY

📅 Date Range: 2000-01-03 to 2025-11-16
🎾 Total Matches: 66,681
🏆 Tournaments: 268
🎮 Players: 1779

🎾 Matches by Year:
   2000: 2,874 matches
   2001: 2,979 matches
   2002: 2,700 matches
   2003: 2,698 matches
   2004: 2,782 matches
   2005: 2,815 matches
   2006: 2,793 matches
   2007: 2,712 matches
   2008: 2,565 matches
   2009: 2,607 matches
   2010: 2,562 matches
   2011: 2,565 matches
   2012: 2,508 matches
   2013: 2,538 matches
   2014: 2,448 matches
   2015: 2,518 matches
   2016: 2,520 matches
   2017: 2,528 matches
   2018: 2,568 matches
   2019: 2,497 matches
   2020: 1,231 matches
   2021: 2,402 matches
   2022: 2,545 matches
   2023: 2,607 matches
   2024: 2,631 matches
   2025: 2,488 matches

🏟️ Surface Distribution:
   Hard: 36,216 matches (54.3%)
   Clay: 21,389 matches (32.1%)
   Grass: 7,

In [3]:
print("\n🎮 STEP 2: CALCULATING ELO RATINGS")
print("="*60)
print("⏳ This will take 10-15 minutes... (grab coffee! ☕)")
print()

# Calculate ELO on ALL data (need continuity from train to test)
all_data = pd.concat([train_df, test_df]).sort_values('Date').reset_index(drop=True)
all_with_elo, elo_calc = calculate_elo_for_dataframe(all_data)

# Split back into train/test
train_with_elo = all_with_elo[all_with_elo['Year'] < 2025].copy()
test_with_elo = all_with_elo[all_with_elo['Year'] >= 2025].copy()

print(f"\n✅ ELO calculated!")
print(f"   Training ELO range: {train_with_elo['elo_1'].min():.0f} - {train_with_elo['elo_1'].max():.0f}")
print(f"   Test ELO range: {test_with_elo['elo_1'].min():.0f} - {test_with_elo['elo_1'].max():.0f}")


🎮 STEP 2: CALCULATING ELO RATINGS
⏳ This will take 10-15 minutes... (grab coffee! ☕)

🎮 Calculating ELO ratings...
   This may take a few minutes for large datasets...
   Processed 5,000 matches...
   Processed 10,000 matches...
   Processed 15,000 matches...
   Processed 20,000 matches...
   Processed 25,000 matches...
   Processed 30,000 matches...
   Processed 35,000 matches...
   Processed 40,000 matches...
   Processed 45,000 matches...
   Processed 50,000 matches...
   Processed 55,000 matches...
   Processed 60,000 matches...
   Processed 65,000 matches...
✅ ELO calculation complete!
   Average ELO: 1635.5
   ELO range: 1299.6 - 2311.6

✅ ELO calculated!
   Training ELO range: 1300 - 2312
   Test ELO range: 1388 - 2176


In [4]:
print("\n🔧 STEP 3: ENGINEERING FEATURES")
print("="*60)
print("⏳ This will take another 10-15 minutes...")
print()

# Engineer features for all data
all_features = engineer_all_features(all_with_elo)

# Split back
train_features = all_features[all_features['Year'] < 2025].copy()
test_features = all_features[all_features['Year'] >= 2025].copy()

print(f"\n✅ Features engineered!")
print(f"   Total columns before cleanup: {len(train_features.columns)}")

# CRITICAL: Keep metadata columns for evaluation later
# These won't be used for training, but we need them for analysis
metadata_cols = [
    'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 
    'Player_1', 'Player_2', 'Winner', 'Score', 'Best of',
    'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2',
    'Year', 'target'
]

# Check which metadata columns exist
existing_metadata = [col for col in metadata_cols if col in train_features.columns]
print(f"   Metadata columns found: {len(existing_metadata)}")

# Clean up features
print("\n🧹 Cleaning features...")

# Get all non-numeric columns
non_numeric_cols = train_features.select_dtypes(include=['object']).columns.tolist()

# Determine which non-numeric columns to drop (exclude metadata and target)
cols_to_drop = [col for col in non_numeric_cols 
                if col not in existing_metadata and col != 'target']

if cols_to_drop:
    print(f"   Dropping {len(cols_to_drop)} non-numeric non-metadata columns")
    train_features = train_features.drop(columns=cols_to_drop)
    test_features = test_features.drop(columns=cols_to_drop)

# Handle numeric columns only (don't touch metadata strings)
numeric_cols = train_features.select_dtypes(include=[np.number]).columns.tolist()

# Replace infinities
train_features[numeric_cols] = train_features[numeric_cols].replace([np.inf, -np.inf], np.nan)
test_features[numeric_cols] = test_features[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Fill NaN
train_features[numeric_cols] = train_features[numeric_cols].fillna(0)
test_features[numeric_cols] = test_features[numeric_cols].fillna(0)

print("✅ Features cleaned!")
print(f"   Total columns after cleanup: {len(train_features.columns)}")
print(f"   Numeric feature columns: {len(numeric_cols)}")
print(f"   Metadata columns preserved: {len(existing_metadata)}")

# Show which metadata columns we have
if existing_metadata:
    print(f"\n📋 Preserved for analysis:")
    for col in existing_metadata[:10]:  # Show first 10
        print(f"   • {col}")
    if len(existing_metadata) > 10:
        print(f"   ... and {len(existing_metadata) - 10} more")

# Save
os.makedirs('../data/processed', exist_ok=True)
train_features.to_csv('../data/processed/train_features.csv', index=False)
test_features.to_csv('../data/processed/test_features.csv', index=False)
print("\n✅ Feature data saved!")


🔧 STEP 3: ENGINEERING FEATURES
⏳ This will take another 10-15 minutes...


🔧 FEATURE ENGINEERING

🔧 Creating basic features...
✅ Basic features created
📊 Calculating recent form (last 10 matches)...
   Processed 5,000 matches...
   Processed 10,000 matches...
   Processed 15,000 matches...
   Processed 20,000 matches...
   Processed 25,000 matches...
   Processed 30,000 matches...
   Processed 35,000 matches...
   Processed 40,000 matches...
   Processed 45,000 matches...
   Processed 50,000 matches...
   Processed 55,000 matches...
   Processed 60,000 matches...
   Processed 65,000 matches...
✅ Recent form calculated
🤝 Calculating head-to-head records...
   Processed 10,000 matches...
   Processed 20,000 matches...
   Processed 30,000 matches...
   Processed 40,000 matches...
   Processed 50,000 matches...
   Processed 60,000 matches...
✅ Head-to-head records calculated
🏟️ Calculating surface-specific stats...
   Processed 10,000 matches...
   Processed 20,000 matches...
   Processed

In [5]:
print("\n🏋️ STEP 4: TRAINING XGBOOST MODEL")
print("="*60)

# Split training data for validation
train_2023 = train_features[train_features['Year'] < 2024].copy()
val_2024 = train_features[train_features['Year'] == 2024].copy()

print(f"   Training: {len(train_2023):,} matches (2000-2023)")
print(f"   Validation: {len(val_2024):,} matches (2024)")

# Initialize and train model
model = TennisPredictionModel(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=6,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

metrics = model.train(train_2023, val_2024, verbose=False)

print(f"\n✅ Training complete!")
print(f"   Training Accuracy: {metrics['train_accuracy']:.1%}")
print(f"   Validation Accuracy: {metrics['val_accuracy']:.1%}")

# Save model
os.makedirs('../models', exist_ok=True)
model.save_model('../models/xgboost_tennis_model.pkl')


🏋️ STEP 4: TRAINING XGBOOST MODEL
   Training: 61,562 matches (2000-2023)
   Validation: 2,631 matches (2024)

🏋️ TRAINING XGBOOST MODEL

📊 Training set: 61,562 matches
🔧 Features: 32
📊 Validation set: 2,631 matches

✅ Validation Accuracy: 0.6842 (68.42%)

🔝 Top 10 Most Important Features:
   odds_implied_prob_1: 0.2936
   odds_diff: 0.1690
   odds_implied_prob_2: 0.1615
   rank_diff: 0.0165
   is_grand_slam: 0.0150
   elo_diff: 0.0148
   rank_2_log: 0.0140
   h2h_wins_1: 0.0136
   elo_1: 0.0135
   points_diff: 0.0134


✅ Training complete!
   Training Accuracy: 73.1%
   Validation Accuracy: 68.4%
✅ Model saved to ../models/xgboost_tennis_model.pkl


In [6]:
print("\n🎾 STEP 5: MAKING PREDICTIONS ON 2025")
print("="*60)

# Make predictions (model only uses feature columns internally)
test_features['prediction'] = model.predict(test_features)
test_features['prediction_proba'] = model.predict(test_features, return_proba=True)

# The test_features DataFrame still has ALL columns (Surface, Tournament, etc.)
# Only the model.predict() method uses the feature columns

# Verify we have the columns we need
print(f"✅ Predictions made!")
print(f"   Metadata preserved: {', '.join(['Surface', 'Tournament', 'Round']) if all(col in test_features.columns for col in ['Surface', 'Tournament', 'Round']) else 'Some columns missing'}")

# Save predictions
os.makedirs('../data/predictions', exist_ok=True)
test_features.to_csv('../data/predictions/predictions_2025.csv', index=False)
print("✅ Predictions saved!")


🎾 STEP 5: MAKING PREDICTIONS ON 2025
✅ Predictions made!
   Metadata preserved: Surface, Tournament, Round
✅ Predictions saved!


In [7]:
print("\n STEP 6: EVALUATION RESULTS")
print("="*60)

# Overall accuracy
overall_accuracy = (test_features['prediction'] == test_features['target']).mean()
total_correct = (test_features['prediction'] == test_features['target']).sum()
total_matches = len(test_features)

print(f"\n🎯 Overall 2025 Accuracy: {overall_accuracy:.1%}")
print(f"   Correct: {total_correct:,} / {total_matches:,} matches")

# Check what columns we have
metadata_cols = ['Surface', 'Tournament', 'Round', 'Series']
available_metadata = [col for col in metadata_cols if col in test_features.columns]

if len(available_metadata) > 0:
    print(f"\n✅ Available for analysis: {', '.join(available_metadata)}")
    
    # By surface
    if 'Surface' in test_features.columns:
        print(f"\n🏟️ Accuracy by Surface:")
        for surface in sorted(test_features['Surface'].unique()):
            surface_data = test_features[test_features['Surface'] == surface]
            surface_acc = (surface_data['prediction'] == surface_data['target']).mean()
            print(f"   {surface}: {surface_acc:.1%} ({len(surface_data)} matches)")
    
    # By tournament type
    if 'is_grand_slam' in test_features.columns:
        print(f"\n🏆 Accuracy by Tournament Type:")
        grand_slams = test_features[test_features['is_grand_slam'] == 1]
        if len(grand_slams) > 0:
            gs_acc = (grand_slams['prediction'] == grand_slams['target']).mean()
            print(f"   Grand Slams: {gs_acc:.1%} ({len(grand_slams)} matches)")
        
        if 'is_masters' in test_features.columns:
            masters = test_features[test_features['is_masters'] == 1]
            if len(masters) > 0:
                masters_acc = (masters['prediction'] == masters['target']).mean()
                print(f"   Masters: {masters_acc:.1%} ({len(masters)} matches)")
else:
    print(f"\n⚠️ Metadata columns were removed during feature engineering")
    print(f"   Can only show overall accuracy")

print("\n" + "="*60)


📊 STEP 6: EVALUATION RESULTS

🎯 Overall 2025 Accuracy: 67.1%
   Correct: 1,670 / 2,488 matches

✅ Available for analysis: Surface, Tournament, Round, Series

🏟️ Accuracy by Surface:
   Clay: 67.4% (733 matches)
   Grass: 68.3% (287 matches)
   Hard: 66.8% (1468 matches)

🏆 Accuracy by Tournament Type:
   Grand Slams: 74.2% (480 matches)
   Masters: 65.6% (748 matches)



In [8]:
print("\n🏆 WIMBLEDON 2025 ANALYSIS")
print("="*60)

# Check if we have Tournament column
if 'Tournament' not in test_features.columns:
    print("\n⚠️ Tournament column not available")
    print("   Cannot analyze Wimbledon specifically")
else:
    wimbledon = test_features[test_features['Tournament'] == 'Wimbledon'].copy()
    
    if len(wimbledon) > 0:
        wimb_accuracy = (wimbledon['prediction'] == wimbledon['target']).mean()
        wimb_correct = (wimbledon['prediction'] == wimbledon['target']).sum()
        
        print(f"\n✅ Overall Accuracy: {wimb_accuracy:.1%} ({wimb_correct}/{len(wimbledon)} matches)")
        
        # By round (if available)
        if 'Round' in wimbledon.columns:
            print(f"\n📊 Accuracy by Round:")
            for round_name in ['1st Round', '2nd Round', '3rd Round', '4th Round', 
                               'Quarterfinals', 'Semifinals', 'The Final']:
                round_data = wimbledon[wimbledon['Round'] == round_name]
                if len(round_data) > 0:
                    round_acc = (round_data['prediction'] == round_data['target']).mean()
                    print(f"   {round_name}: {round_acc:.1%} ({len(round_data)} matches)")
        
        # Final match (if available)
        if 'Round' in wimbledon.columns:
            final = wimbledon[wimbledon['Round'] == 'The Final']
            if len(final) > 0:
                print(f"\n🏆 WIMBLEDON 2025 FINAL:")
                final_row = final.iloc[0]
                if 'Player_1' in final_row and 'Player_2' in final_row:
                    print(f"   {final_row['Player_1']} vs {final_row['Player_2']}")
                    print(f"   Winner: {final_row['Winner']}")
                    predicted_winner = final_row['Player_1'] if final_row['prediction'] == 1 else final_row['Player_2']
                    print(f"   Predicted: {predicted_winner}")
                    print(f"   Correct: {'✅ YES' if final_row['prediction'] == final_row['target'] else '❌ NO'}")
                    print(f"   Confidence: {final_row['prediction_proba']:.1%}")
    else:
        print("\n⚠️ No Wimbledon 2025 data found")

print("\n" + "="*60)


🏆 WIMBLEDON 2025 ANALYSIS

✅ Overall Accuracy: 71.3% (87/122 matches)

📊 Accuracy by Round:
   1st Round: 68.9% (61 matches)
   2nd Round: 65.6% (32 matches)
   3rd Round: 81.2% (16 matches)
   4th Round: 83.3% (6 matches)
   Quarterfinals: 100.0% (4 matches)
   Semifinals: 100.0% (2 matches)
   The Final: 0.0% (1 matches)

🏆 WIMBLEDON 2025 FINAL:
   Alcaraz C. vs Sinner J.
   Winner: Sinner J.
   Predicted: Alcaraz C.
   Correct: ❌ NO
   Confidence: 63.3%



In [9]:
print("\n PIPELINE COMPLETE!")
print("="*60)
print(f"\n📊 Final Results:")
print(f"   Overall 2025 Accuracy: {overall_accuracy:.1%}")
print(f"   Total Predictions: {total_matches:,}")
print(f"   Correct Predictions: {total_correct:,}")
print(f"\n🎯 Comparison:")
print(f"   Random Guessing: 50.0%")
print(f"   Our Model: {overall_accuracy:.1%}")
print(f"   Typical Betting Odds: ~70-72%")
print("\n✅ All done! Check the data/predictions/ folder for results!")
print("="*60)


🎉 PIPELINE COMPLETE!

📊 Final Results:
   Overall 2025 Accuracy: 67.1%
   Total Predictions: 2,488
   Correct Predictions: 1,670

🎯 Comparison:
   Random Guessing: 50.0%
   Our Model: 67.1%
   Typical Betting Odds: ~70-72%

✅ All done! Check the data/predictions/ folder for results!
